In [ ]:
pip install ortools

In [1]:
import pandas as pd
import sys 
import os 
import numpy as np
from datetime import datetime

In [2]:
client_to_client = pd.read_csv('../data/distance_matrix_client_to_client.csv')
inter_to_client = pd.read_csv('../data/distance_matrix_inter_to_client.csv')

In [3]:
client_to_client

,ID Client 1,ID Client 2,geometry_client_1,geometry_client_2,distance_car,duration_car,distance_bike,duration_bike
0,559475456,559277088,POINT (1.3756893 48.7210523),POINT (1.3756893 48.7210523),1 m,1 min,1 m,1 min
1,559475456,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13.8 km,18 mins,12.1 km,44 mins
2,559277088,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13.8 km,18 mins,12.1 km,44 mins
3,559475456,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5.9 km,8 mins,4.2 km,18 mins
4,559277088,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5.9 km,8 mins,4.2 km,18 mins
...,...,...,...,...,...,...,...,...
6898,570590313,621952854,POINT (1.3579211 48.7447023),POINT (1.351704 48.7428046),0.8 km,2 mins,0.6 km,2 mins
6899,167937489,621952854,POINT (1.3475838 48.7428036),POINT (1.351704 48.7428046),0.5 km,2 mins,0.5 km,1 min
6900,474721965,621952854,POINT (1.3706525 48.7406195),POINT (1.351704 48.7428046),2.3 km,7 mins,2.3 km,11 mins
6901,751167409,621952854,POINT (1.3388591 48.7293302),POINT (1.351704 48.7428046),2.4 km,6 mins,3.5 km,12 mins


In [4]:
inter_to_client

,ID Intervenant,ID Client,geometry_intervenant,Permis,geometry_client,distance,duration
0,162858075,559475456,POINT (1.3746477 48.7222691),1.0,POINT (1.3756893 48.7210523),375,83
1,78007018,559475456,POINT (1.351054 48.689927),1.0,POINT (1.3756893 48.7210523),5454,497
2,856089133,559475456,POINT (1.5230773 48.6454179),0.0,POINT (1.3756893 48.7210523),17631,3520
3,842621873,559475456,POINT (1.3798422 48.728863),1.0,POINT (1.3756893 48.7210523),1732,250
4,838320706,559475456,POINT (1.3919715 48.7385156),1.0,POINT (1.3756893 48.7210523),3884,326
...,...,...,...,...,...,...,...
2827,818696864,621952854,POINT (1.3579211 48.7447023),1.0,POINT (1.351704 48.7428046),805,149
2828,813991780,621952854,POINT (1.0731951 48.6927148),1.0,POINT (1.351704 48.7428046),23526,1463
2829,739888851,621952854,POINT (1.303906 48.75186),1.0,POINT (1.351704 48.7428046),5135,362
2830,78012267,621952854,POINT (1.3719853 48.7292064),1.0,POINT (1.351704 48.7428046),3234,530


In [5]:
parent_directory = os.path.dirname(os.getcwd())
sys.path.append(parent_directory)  

from utils.utils import load_data
from utils.utils import parse_duration, determine_time_window, prestation_duration

In [6]:
data = load_data()

df = data['JAN24']

In [7]:
for i in range(len(df)):
    heure_debut = df.at[i, 'Heure de début']
    if df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 10:
        df.at[i, 'Prestation'] = 'PDJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 14 and heure_debut.hour>12 :
        df.at[i, 'Prestation'] = 'DEJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS':
        df.at[i, 'Prestation'] = 'DIN'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour < 12:
        df.at[i, 'Prestation'] = 'TOILETTE_MAT'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour > 12:
        df.at[i, 'Prestation'] = 'TOILETTE_SOIR'
    

In [8]:
data_day = df[df['Date']=='2024-01-01'].copy()

In [9]:
prestation_duration_list = []
for i in range(len(data_day)):
    h1 = data_day.iloc[i].loc['Heure de début']
    h2 = data_day.iloc[i].loc['Heure de fin']

    prestation_duration_list.append(prestation_duration(h1,h2))

data_day['Prestation_duration'] = prestation_duration_list

In [10]:
data_day['time_window'] = data_day['Prestation'].apply(determine_time_window)

In [11]:
data_day

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation,Prestation_duration,time_window
0,559475456,162858075,2024-01-01,07:15:00,07:45:00,PDJ,30.0,"(7, 9)"
1,559277088,162858075,2024-01-01,07:45:00,08:30:00,TOILETTE_MAT,45.0,"(7, 10)"
2,87852633,78007018,2024-01-01,07:45:00,08:30:00,TOILETTE_MAT,45.0,"(7, 10)"
3,243033408,810259688,2024-01-01,07:45:00,08:15:00,TOILETTE_MAT,30.0,"(7, 10)"
4,814940942,710283561,2024-01-01,07:45:00,09:20:00,TOILETTE_MAT,95.0,"(7, 10)"
...,...,...,...,...,...,...,...,...
61,714782168,78007018,2024-01-01,19:00:00,20:00:00,DIN,60.0,"(19, 21)"
62,814940942,810259688,2024-01-01,19:00:00,20:00:00,TOILETTE_SOIR,60.0,"(18, 20)"
63,530138806,480302361,2024-01-01,19:15:00,19:45:00,DIN,30.0,"(19, 21)"
64,803656603,856089133,2024-01-01,19:55:00,20:25:00,TOILETTE_SOIR,30.0,"(18, 20)"


In [14]:
intervention_intervention_matrix = np.zeros(len(data_day)+len(data_day['ID Intervenant'].unique()), len(data_day)+len(data_day['ID Intervenant'].unique()))

for i in range(0,len(data_day)+len(data_day['ID Intervenant'].unique())):
    for j in range(i,len(data_day)): 
        
        if (i<len(len(data_day['ID Intervenant'].unique()))) & (j<len(len(data_day['ID Intervenant'].unique())) & (i!=j)):
            intervention_intervention_matrix[i,j] = 10000
        
        if (i<len(data_day['ID Intervenant'].unique()) & (j>=len(data_day['ID Intervenant'].unique()))):
            intervenant_i = data_day['ID Intervenant'].unique().iloc[i]
            intervention_j = data_day['ID Client'].iloc[j-len(data_day['ID Intervenant'].unique())]

            row_1 = inter_to_client[(inter_to_client['ID Client'] == intervention_j) & (inter_to_client['ID Intervenant'] == intervenant_i)]

            if not row_1.empty:
                distance = row_1['duration'].iloc[0]
                intervention_intervention_matrix[i, j] = distance/60
            
            else:
                    print(intervenant_i, intervention_j)
                    intervention_intervention_matrix[i, j] = np.nan


    
        if (i>=len(len(data_day['ID Intervenant'].unique()))) & (j>=len(len(data_day['ID Intervenant'].unique()))):
            intervention_i = data_day['ID Client'].iloc[i-len(data_day['ID Intervenant'].unique())]
            intervention_j = data_day['ID Client'].iloc[j-len(data_day['ID Intervenant'].unique())]

            if intervention_i == intervention_j:
                intervention_intervention_matrix[i,j] = 0
            
            else:
                row_1 = client_to_client[(client_to_client['ID Client 1'] == intervention_i) & (client_to_client['ID Client 2'] == intervention_j)]
                row_2 = client_to_client[(client_to_client['ID Client 1'] == intervention_j) & (client_to_client['ID Client 2'] == intervention_i)]
                
                if not row_1.empty:
                    distance = row_1['duration_car'].iloc[0]
                    intervention_intervention_matrix[i, j] = parse_duration(distance)

                elif not row_2.empty:
                    distance = row_2['duration_car'].iloc[0]
                    intervention_intervention_matrix[i, j] = parse_duration(distance)

                else:
                    print(intervention_i, intervention_j)
                    intervention_intervention_matrix[i, j] = np.nan  
                
for i in range(len(data_day)):
    for j in range(i):
        intervention_intervention_matrix[i, j] = intervention_intervention_matrix[j, i]

TypeError: Cannot interpret '74' as a data type

In [13]:
intervention_intervention_matrix

array([[ 0.,  1., 18., ...,  8.,  2.,  4.],
       [ 1.,  0., 18., ...,  8.,  2.,  4.],
       [18., 18.,  0., ..., 22., 16., 19.],
       ...,
       [ 8.,  8., 22., ...,  0., 10.,  5.],
       [ 2.,  2., 16., ..., 10.,  0.,  5.],
       [ 4.,  4., 19., ...,  5.,  5.,  0.]])

In [ ]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    #data["time_matrix"] = intervention_intervention_matrix
    #data["time_windows"] = data_day['time_window']
    data["time_matrix"] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data["time_windows"] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 4),  # 7
        (5, 10),  # 8
        (0, 3),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 8),  # 14
        (10, 15),  # 15
        (11, 15),  # 16
    ]
    
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        1000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
        )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()